In [2]:
# pip install selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from tqdm.auto import tqdm 
import time
import re
import pandas as pd


c:\Users\piano\anaconda3\envs\hyojung_2015479\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
artist_links = pd.read_csv('moma_artists_final.csv')
display(artist_links.head()) 
display(artist_links.shape)

,artist_link
0,https://artfacts.net/artist/elinor-bellingham-...
1,https://artfacts.net/artist/frank-brangwyn/30490
2,https://artfacts.net/artist/stephen-conroy/1625
3,https://artfacts.net/artist/hans-feibusch/30758
4,https://artfacts.net/artist/james-flewitt-mull...


(28489, 1)

In [8]:
artist_links = artist_links.dropna()
artist_links.shape

(28489, 1)

In [9]:
artist_links = pd.read_csv('artists_split1.csv')
artist_links.shape

(5000, 1)

In [10]:
artist_links["artist_link"] = artist_links["artist_link"].apply(lambda x: x + "/exhibitions")

exhibitions = artist_links.copy()

In [11]:
hrefs = exhibitions['artist_link'].dropna()
hrefs

0       https://artfacts.net/artist/elinor-bellingham-...
1       https://artfacts.net/artist/frank-brangwyn/304...
2       https://artfacts.net/artist/stephen-conroy/162...
3       https://artfacts.net/artist/hans-feibusch/3075...
4       https://artfacts.net/artist/james-flewitt-mull...
                              ...                        
4995    https://artfacts.net/artist/alberto-biasi/2543...
4996    https://artfacts.net/artist/milivoj-bijelic/25...
4997    https://artfacts.net/artist/simon-bogojevic-na...
4998    https://artfacts.net/artist/zoe-borelli/25365/...
4999    https://artfacts.net/artist/zdravko-bregovac/1...
Name: artist_link, Length: 5000, dtype: object

In [12]:
# def collect_museum_collections_info():
#     try:
#         section_header = driver.find_element(By.XPATH, "//h3[text()='Museum Collections']")
#         artist_card_list = section_header.find_element(By.XPATH, "./ancestor::div[contains(@class, 'app-js-components-ArtistCard-ArtistCard__list')]")
#         section_containers = artist_card_list.find_elements(By.XPATH, ".//div[contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__container')]")

#         all_items = []
#         pending_items = []

#         for container in section_containers:
#             labels_and_items = container.find_elements(By.XPATH, ".//div[contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__label') or contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__item')]")

#             for element in labels_and_items:
#                 if "app-js-components-LabeledItemList-LabeledItemList__label" in element.get_attribute("class"):
#                     current_country = element.text.strip()
                    
#                     # Assign the current label to all pending items
#                     for item in pending_items:
#                         item['Country'] = current_country
#                         all_items.append(item)
#                     pending_items.clear()
#                 elif "app-js-components-LabeledItemList-LabeledItemList__item" in element.get_attribute("class"):
#                     li_elements = element.find_elements(By.TAG_NAME, "li")

#                     for li_element in li_elements:
#                         span_elements = li_element.find_elements(By.TAG_NAME, "span")
#                         link = li_element.find_element(By.TAG_NAME, "a").get_attribute('href')

#                         if len(span_elements) >= 2:
#                             name = span_elements[0].text.strip()
#                             city = span_elements[1].text.strip()
#                         elif len(span_elements) == 1:
#                             name = span_elements[0].text.strip()
#                             city = 'N/A'
#                         else:
#                             name = li_element.text.strip()
#                             city = 'N/A'

#                         pending_items.append({
#                             'Country': None,  # Placeholder for later label assignment
#                             'Name': name,
#                             'City': city,
#                             'url': link
#                         })

#         # Assign the last known label to any remaining pending items
#         for item in pending_items:
#             item['Country'] = current_country
#             all_items.append(item)

#         return all_items
#     except Exception as e:
#         return []

def collect_museum_collections_info():
    try:
        section_header = driver.find_element(By.XPATH, "//h3[text()='Museum Collections']")
        artist_card_list = section_header.find_element(By.XPATH, "./ancestor::div[contains(@class, 'app-js-components-ArtistCard-ArtistCard__list')]")
        section_containers = artist_card_list.find_elements(By.XPATH, ".//div[contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__container')]")

        all_items = []

        for container in section_containers:
            items = container.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__item")
            for item in items:
                name = item.text.strip()
                url = item.find_element(By.TAG_NAME, "a").get_attribute('href')
                all_items.append({'Name': name, 'URL': url})

        return all_items
    except Exception as e:
        return []

# def collect_section_info(section_title):
#     try:
#         section = driver.find_element(By.XPATH, f"//div[h3[text()='{section_title}']]")
#         items_container = section.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__container")
        
#         all_items = []
#         pending_items = []
#         current_year = None
        
#         for container in items_container:
#             labels_and_items = container.find_elements(By.XPATH, ".//div[contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__label') or contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__item')]")
            
#             for element in labels_and_items:
#                 if "app-js-components-LabeledItemList-LabeledItemList__label" in element.get_attribute("class"):
#                     current_year = element.text.strip()
                    
#                     # Add pending items with the current year
#                     for item in pending_items:
#                         item['Year'] = current_year
#                         all_items.append(item)
#                     pending_items.clear()
                    
#                 elif "app-js-components-LabeledItemList-LabeledItemList__item" in element.get_attribute("class"):
#                     li_elements = element.find_elements(By.TAG_NAME, "li")
#                     for li_element in li_elements:
#                         a_tags = li_element.find_elements(By.TAG_NAME, "a")
#                         if len(a_tags) >= 4:
#                             try:
#                                 name = a_tags[0].get_attribute('innerText')
#                             except:
#                                 name = a_tags[0].text
#                             try:
#                                 institution = a_tags[1].get_attribute('innerText')
#                             except:
#                                 institution = a_tags[1].text
#                             try:
#                                 city = a_tags[2].get_attribute('innerText')
#                             except:
#                                 city = a_tags[2].text
#                             try:
#                                 country = a_tags[3].get_attribute('innerText')
#                             except:
#                                 country = a_tags[3].text
#                             pending_items.append({
#                                 'Year': current_year,  # Will be replaced when the next label is found
#                                 'Name': name,
#                                 'Institution': institution,
#                                 'City': city,
#                                 'Country': country,
#                                 'url': a_tags[0].get_attribute('href')
#                             })
        
#         # Add any remaining pending items
#         for item in pending_items:
#             item['Year'] = current_year
#             all_items.append(item)
        
#         return all_items
#     except Exception as e:
#         return []

def collect_section_info(section_title):
    try:
        section = driver.find_element(By.XPATH, f"//div[h3[text()='{section_title}']]")
        items = section.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__item")
        
        all_items = []
        for item in items:
            name = item.text.strip()
            url = item.find_element(By.TAG_NAME, "a").get_attribute('href')
            all_items.append({'Name': name, 'URL': url})
        
        return all_items
    except Exception as e:
        return []

def collect_dealer_directory_info():
    try:
        section = driver.find_element(By.XPATH, "//div[h3[text()='Dealer Directory']]")
        items_container = section.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__container")

        all_items = []
        for container in items_container:
            labels = container.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__label")
            items = container.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__item")
            for label, item in zip(labels, items):
                all_items.append({
                    'Country': label.text,
                    'Name': item.text,
                    'url': item.find_element(By.TAG_NAME, "a").get_attribute('href')
                })
        return all_items
    except Exception as e:
        return []

def collect_catalogs_info():
    try:
        section = driver.find_element(By.XPATH, "//div[h3[text()='Catalogs']]")
        items_container = section.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__container")

        all_items = []
        for container in items_container:
            labels = container.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__label")
            items = container.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__item")
            for label, item in zip(labels, items):
                all_items.append({
                    'Year': label.text,
                    'Name': item.text,
                    'url': item.find_element(By.TAG_NAME, "a").get_attribute('href')
                })
        return all_items
    except Exception as e:
        return []
    

def collect_exhibitions_info(section_title):
    try:
        section = driver.find_element(By.XPATH, f"//div[h3[text()='{section_title}']]")
        items_container = section.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__container")
        
        all_items = []
        pending_items = []
        current_year = None
        
        for container in items_container:
            labels_and_items = container.find_elements(By.XPATH, ".//div[contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__label') or contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__item')]")
            
            for element in labels_and_items:
                if "app-js-components-LabeledItemList-LabeledItemList__label" in element.get_attribute("class"):
                    current_year = element.text.strip()
                    
                    # Add pending items with the current year
                    for item in pending_items:
                        item['Year'] = current_year
                        all_items.append(item)
                    pending_items.clear()
                    
                elif "app-js-components-LabeledItemList-LabeledItemList__item" in element.get_attribute("class"):
                    li_elements = element.find_elements(By.TAG_NAME, "li")
                    for li_element in li_elements:
                        a_tags = li_element.find_elements(By.TAG_NAME, "a")
                        if len(a_tags) >= 4:
                            try:
                                name = a_tags[0].get_attribute('innerText')
                            except:
                                name = a_tags[0].text
                            try:
                                institution = a_tags[1].get_attribute('innerText')
                            except:
                                institution = a_tags[1].text
                            try:
                                city = a_tags[2].get_attribute('innerText')
                            except:
                                city = a_tags[2].text
                            try:
                                country = a_tags[3].get_attribute('innerText')
                            except:
                                country = a_tags[3].text

                            # Group Exhibition Type
                            exhibition_type = 'Group Exhibition' if 'Group Exhibition' in li_element.text else 'Solo Exhibition'

                            pending_items.append({
                                'Year': current_year,  # Will be replaced when the next label is found
                                'Name': name,
                                'Institution': institution,
                                'City': city,
                                'Country': country,
                                'Type': exhibition_type,  # Group or Solo Exhibition
                                'url': a_tags[0].get_attribute('href')
                            })
        
        # Add any remaining pending items
        for item in pending_items:
            item['Year'] = current_year
            all_items.append(item)
        
        return all_items
    except Exception as e:
        return []





In [16]:
# 로그인 세션 생성 함수
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--incognito')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')

    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    driver.get(hrefs[0])
    time.sleep(2)  # 페이지가 로드되도록 대기

    # 로그인 옵션 클릭
    login_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[text()='Login']")))
    driver.execute_script("arguments[0].click();", login_link)

    # 로그인 세부사항 입력 및 제출
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "email"))).send_keys('wkyoung2@gmail.com')
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "password"))).send_keys('!BAwk123123')

    # login 버튼 선택 및 클릭
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "app-js-components-AuthenticationModal-AuthenticationModal__button"))
    )
    driver.execute_script("arguments[0].click();", login_button)

    time.sleep(5)  # 로그인 후 페이지가 로드되도록 충분한 시간 대기
    return driver

# 데이터 수집 함수
def collect_data(driver, hrefs, all_info):
    for href in hrefs:
        driver.get(href)

        # 특정 요소가 로드될 때까지 대기 (최대 5초)
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="application-root"]/div/main/div/div[2]/div/div[2]'))
            )
        except TimeoutException:
            print(f"{href} - 요소를 찾을 수 없음, 다음 URL로 이동")
            continue

        # 페이지 끝까지 스크롤
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # 아티스트 이름 불러오기
        label = driver.find_element(By.XPATH, '//*[@id="application-root"]/div/main/div/div[1]/div[1]/div/h1').text

        # 각 섹션의 정보 수집
        museum_collections = collect_museum_collections_info() or []
        biennials = collect_section_info('Biennials') or []
        art_fairs = collect_section_info('Art fairs') or []
        exhibitions = collect_section_info('Exhibitions') or []
        dealer_directory = collect_section_info('Dealer Directory') or []
        catalogs = collect_section_info('Catalogs') or []

        # 수집된 데이터를 all_info 리스트에 추가
        for collection in museum_collections:
            all_info.append({'Artist': label, 'Type': 'Museum Collections', **collection})

        for biennial in biennials:
            all_info.append({'Artist': label, 'Type': 'Biennials', **biennial})

        for art_fair in art_fairs:
            all_info.append({'Artist': label, 'Type': 'Art Fairs', **art_fair})

        for exhibition in exhibitions:
            all_info.append({'Artist': label, 'Type': 'Exhibitions', **exhibition})

        for dealer in dealer_directory:
            all_info.append({'Artist': label, 'Type': 'Dealer Directory', **dealer})

        for catalog in catalogs:
            all_info.append({'Artist': label, 'Type': 'Catalogs', **catalog})

        print(f"{href} collected: {len(museum_collections) + len(biennials) + len(art_fairs) + len(exhibitions) + len(dealer_directory) + len(catalogs)} items")


driver = create_driver()
all_info = []  # 각 배치에 대한 데이터를 저장할 리스트

collect_data(driver, hrefs, all_info)  # 기존에 정의된 데이터 수집 함수 사용

driver.quit()

https://artfacts.net/artist/elinor-bellingham-smith/102823/exhibitions collected: 6 items
https://artfacts.net/artist/frank-brangwyn/30490/exhibitions collected: 11 items
https://artfacts.net/artist/stephen-conroy/1625/exhibitions collected: 15 items
https://artfacts.net/artist/hans-feibusch/30758/exhibitions collected: 9 items
https://artfacts.net/artist/james-flewitt-mullock/224412/exhibitions - 요소를 찾을 수 없음, 다음 URL로 이동
https://artfacts.net/artist/william-russel-flint/85849/exhibitions collected: 10 items
https://artfacts.net/artist/stanhope-alexander-forbes/57186/exhibitions collected: 10 items
https://artfacts.net/artist/harry-holland/21900/exhibitions collected: 13 items
https://artfacts.net/artist/laura-knight/32231/exhibitions collected: 13 items


KeyboardInterrupt: 

In [17]:
all_info

[{'Artist': 'Elinor Bellingham-Smith',
  'Type': 'Museum Collections',
  'Name': 'Tate Britain\nLondon',
  'URL': 'https://artfacts.net/institution/tate-britain/2235'},
 {'Artist': 'Elinor Bellingham-Smith',
  'Type': 'Museum Collections',
  'Name': 'Newport Museum & Art Gallery\nNewport, Wales',
  'URL': 'https://artfacts.net/institution/newport-museum-art-gallery/20175'},
 {'Artist': 'Elinor Bellingham-Smith',
  'Type': 'Exhibitions',
  'Name': 'Unpopular Culture\nGroup Exhibition\nMead Gallery - Warwick Arts CentreCoventryUnited Kingdom',
  'URL': 'https://artfacts.net/exhibition/unpopular-culture/506913'},
 {'Artist': 'Elinor Bellingham-Smith',
  'Type': 'Exhibitions',
  'Name': 'Suffolk : A Female Focus\nGroup Exhibition\nWolsey Art Gallery & The Room UpstairsIpswichUnited Kingdom',
  'URL': 'https://artfacts.net/exhibition/suffolk-:-a-female-focus/324841'},
 {'Artist': 'Elinor Bellingham-Smith',
  'Type': 'Exhibitions',
  'Name': 'Recent British Drawings\nGroup Exhibition\nInstit

In [18]:
# 수집한 모든 정보를 DataFrame으로 변환
df = pd.DataFrame(all_info)
df
df.to_excel('artists_exhibitions_info2.xlsx', index=False)